<a href="https://colab.research.google.com/github/Athugodage/VK_NLP/blob/main/vk_nlp_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Athugodage/VK_NLP

Cloning into 'VK_NLP'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 23 (delta 2), reused 17 (delta 0), pack-reused 0
Unpacking objects: 100% (23/23), 27.83 KiB | 2.32 MiB/s, done.


In [7]:
!pip install razdel emoji translate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.6/356.6 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 6.6 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-2.6.0-py2.py3-none-any.whl size=351311 sha256=aff511bb573a04c0570eaba84cabf580571927d1d56dbefa8ba34ee752a2d97a
  Stored in directory: /root/.cache/pip/wheels/ea/0b/64/114bc939d0083621aa41521e21be246c888260b8aa21e6c1ad
  Created wheel f

In [59]:
import pandas as pd

df = pd.read_csv('/content/posts.csv', encoding='utf8', delimiter=',')

In [60]:
from razdel import tokenize

df['length'] = df['text'].apply(lambda x: len(list(tokenize(str(x)))))

In [61]:
df = df[df['length'] > 30]

In [62]:
df

,id,text,view_hash,time,url,likes,funny,wow,delight,sadness,bad,views,shares,community_id,post_id,local_time,length
0,1056,С днем защиты детей! В АРТКВАДРАТЕ прошёл конк...,2fb021d8a04054db2b,2 июн в 9:45,https://vk.com/wall-180916324_1490,5,0,0,0,0,0,632,0,180916324,180916324_1490,07-07-2023 15:39:16,131
1,1057,"Сегодня с детьми младшей группы ""Капельки"" про...",2fb021d8a04054db2b,25 июн в 7:36,https://vk.com/wall-216943317_567,17,0,0,0,0,0,632,1,216943317,216943317_567,07-07-2023 15:39:23,56
2,1058,"Мы идём, тик-так , идёмСтрелки крутим день за ...",2fb021d8a04054db2b,25 июн в 7:36,https://vk.com/wall-216943317_566,9,0,0,0,0,0,271,1,216943317,216943317_566,07-07-2023 15:39:23,78
3,1059,Всю-то ночь часы стучат.Ни минуты не молчат!Я ...,2fb021d8a04054db2b,25 июн в 7:36,https://vk.com/wall-216943317_565,15,0,0,0,0,0,249,0,216943317,216943317_565,07-07-2023 15:39:23,75
5,1061,"Лето- время не только для отдыха и игр, но и в...",2fb021d8a04054db2b,23 июн в 5:45,https://vk.com/wall-216943317_558,22,0,0,0,0,0,679,0,216943317,216943317_558,07-07-2023 15:39:23,104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123155,1044,ПРИМИ УЧАСТИЕ В ПРАЗДНИЧНОМ ОНЛАЙН-ФЛЕШМОБЕ Еж...,2fb021d8a04054db2b,3 июл в 14:29,https://vk.com/wall-171462215_4198,0,0,0,0,0,0,88,0,171462215,171462215_4198,07-07-2023 15:39:07,84
123156,1045,ВЫСТАВКА ТВОРЧЕСКИЙ КАЛЕЙДОСКОП 22 июня в Цент...,2fb021d8a04054db2b,3 июл в 12:00,https://vk.com/wall-171462215_4196,3,0,0,0,0,0,100,0,171462215,171462215_4196,07-07-2023 15:39:07,188
123157,1046,ДЕТСКИЕ ДВОРОВЫЕ ПЛОЩАДКИ Не знаете как прове...,2fb021d8a04054db2b,3 июл в 10:00,https://vk.com/wall-171462215_4195,1,0,0,0,0,0,116,0,171462215,171462215_4195,07-07-2023 15:39:07,94
123158,1047,Сегодня в парке «Первомайский» состоялось торж...,2fb021d8a04054db2b,1 июл в 20:00,https://vk.com/wall-171462215_4193,1,0,0,0,0,0,94,0,171462215,171462215_4193,07-07-2023 15:39:07,140


In [63]:
df['text']

0         С днем защиты детей! В АРТКВАДРАТЕ прошёл конк...
1         Сегодня с детьми младшей группы "Капельки" про...
2         Мы идём, тик-так , идёмСтрелки крутим день за ...
3         Всю-то ночь часы стучат.Ни минуты не молчат!Я ...
5         Лето- время не только для отдыха и игр, но и в...
                                ...                        
123155    ПРИМИ УЧАСТИЕ В ПРАЗДНИЧНОМ ОНЛАЙН-ФЛЕШМОБЕ Еж...
123156    ВЫСТАВКА ТВОРЧЕСКИЙ КАЛЕЙДОСКОП 22 июня в Цент...
123157    ДЕТСКИЕ ДВОРОВЫЕ ПЛОЩАДКИ  Не знаете как прове...
123158    Сегодня в парке «Первомайский» состоялось торж...
123160    С 26 по 28 июня 2023 года в Уфе состоится юбил...
Name: text, Length: 84672, dtype: object

In [64]:
print(df['text'].iloc[2])

Мы идём, тик-так , идёмСтрелки крутим день за днём,Чтобы вы не опоздали,Чтобы раньше всех вставали!Прозвоним мы в нужный час -Позовём на завтрак вас!Показать ещёМы идём, тик-так, идём,Но лентяев мы не ждём!Познавательная беседа о времени и о часах в старшей группе .Задачи:-знакомить детей с понятием "часы" и с историей их возникновения -воспитывать любознательность ,бережное отношение ко времени


In [98]:

def clean(text):
  text =  re.sub('[http|www].*', '', text)
  text = re.sub('[#|@][А-яЁёA-z]', '', text)

  return text

df['text'] = df['text'].apply(clean)

In [99]:
df['text'].iloc[19]

' Лучший детский сад Башкирии : муниципальный этап народного голосования Совместно с Министерством образования и науки РБ запускаем муниципальный этап голосования по лучшему Детскому саду Башкирии по мнению жителей региона. Показать ещё Муниципальный этап голосования пройдёт с 21 июня по 18 июля 2023 года в сообществе Башкирия Online. Региональный этап голосования( участники — победители муниципального этапа голосования + лучшие по числу голосов среди занявших 2 и 3 места) пройдёт с 19 июля по 18 августа 2023 года в сообществе ИА" Башинформ". Подведение итогов конкурса состоится в августе 2023 года. Торжественное награждение победителей состоится 27 сентября - в День работников дошкольного образования. Чтобы проголосовать, перейдите по ссылке своего муниципалитета и выберите участника : ГО г. Агидель : '

In [ ]:
from razdel import tokenize
from tqdm import tqdm
import re

tqdm.pandas()

def check_token(word):

  m  = re.search('[а-яё][А-ЯЁ]', word)
  if m:
    word = word[:m.span()[0]+1]+'. '+word[m.span()[0]+1:]

  return word

def text2tokens(text):
  signs = ('!', '?', ',', '.', '"', ')', '(',)
  tokens = ''
  tokenized = list(tokenize(str(text)))

  for token in tokenized:
    token = check_token(token.text)
    if any(sign in token for sign in signs) == False:
      tokens += ' '

    tokens += token

  return tokens


df['text'] = df['text'].progress_apply(text2tokens)


100%|██████████| 84672/84672 [02:29<00:00, 568.06it/s]


In [ ]:
import emoji
from translate import Translator

translator= Translator(to_lang="ru")

def emoji2text(text, emoji, translator):

  demojized = emoji.demojize(text)

  while re.search(':[A-z]+:', text):
    f = re.search(':[A-z]+:', text)

    emoji = f.group(0)
    print(emoji)
    translated = translator.translate(str(emoji))

    text = text[:f.span()[0]] + translated + text[f.span()[1]:]

  return text


df['text'] = df['text'].progress_apply(lambda x: emoji2text(x, emoji, translator))

100%|██████████| 84672/84672 [01:27<00:00, 965.20it/s]


In [ ]:
df['text'].iloc[19]

' Лучший детский сад Башкирии : муниципальный этап народного голосования Совместно с Министерством образования и науки РБ запускаем муниципальный этап голосования по лучшему Детскому саду Башкирии по мнению жителей региона. Показать ещё Муниципальный этап голосования пройдёт с 21 июня по 18 июля 2023 года в сообществе Башкирия Online. Региональный этап голосования( участники — победители муниципального этапа голосования + лучшие по числу голосов среди занявших 2 и 3 места) пройдёт с 19 июля по 18 августа 2023 года в сообществе ИА" Башинформ". Подведение итогов конкурса состоится в августе 2023 года. Торжественное награждение победителей состоится 27 сентября - в День работников дошкольного образования. Чтобы проголосовать, перейдите по ссылке своего муниципалитета и выберите участника : ГО г. Агидель :'

In [105]:
!pip install transformers

In [ ]:
from vk_nlp.sentiment import Sentiment_classification, Emotion_detection, Toxicity_detection

clf = Sentiment_classification()

def classify(text):
  try:
    return clf.classify_text(text)
  except:
    return None


df['sentiment'] = df['text'].progress_apply(classify)

  2%|▏         | 1343/84672 [13:31<15:39:05,  1.48it/s]